# Hospital Service Volumes

In [20]:
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt


1. **`from sqlalchemy import create_engine`**: Imports the **`create_engine`** function from **SQLAlchemy** to connect to a database.
2. **`import pandas as pd`**: Imports the **Pandas** library, which is a powerful tool for data manipulation and analysis.
3. **`import matplotlib.pyplot as plt`**: Imports the **`pyplot`** module from **Matplotlib** for creating visualizations like charts and graphs.


In [22]:
from sqlalchemy import create_engine


server_name = 'DESKTOP-0N0LQTG'
database_name = 'Nova_Scotia_Action_For_Health'


connection_string = f'mssql+pyodbc://{server_name}/{database_name}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'


engine = create_engine(connection_string)


try:
    with engine.connect() as conn:
        print("Connection successful!")
except Exception as e:
    print(f"Connection failed: {str(e)}")


Connection successful!


### Explanation

#### Imports
- **`from sqlalchemy import create_engine`**: Imports the `create_engine` function from sqlalchemy. This function is essential for creating a connection engine that allows Python to interact with a SQL database.

#### Variables
- **`server_name`**: `'DESKTOP-0N0LQTG'` 
- **`database_name`**: `'Nova_Scotia_Action_For_Health'`

#### Constructing Connection String
- Constructs a connection string using Python's f-string formatting:
  - `mssql+pyodbc://`: Specifies the dialect and driver to use (MSSQL with PyODBC driver).
  - `{server_name}/{database_name}`: Specifies the server and database to connect to.
  - `trusted_connection=yes`: Indicates Windows authentication should be used, where the current Windows credentials are used to authenticate.
  - `driver=ODBC+Driver+17+for+SQL+Server`: Specifies the ODBC driver to use for the connection. Adjust this according to the version of the ODBC driver installed on your system.

#### Creating the Engine
- Uses the `create_engine` function from sqlalchemy to create an engine (`engine`) object:
  - The `connection_string` is passed as an argument to `create_engine`, configuring the engine to connect to the specified SQL Server instance and database using the specified authentication method and driver.

#### Testing the Connection
- Attempts to establish a connection to the SQL Server database using the `engine.connect()` method:
  - If the connection is successful (`with` block), it prints "Connection successful!".
  - If an exception occurs during the connection attempt (e.g., authentication error, network issue), the `except` block catches the exception (`Exception as e`), and it prints "Connection failed: " followed by the error message (`str(e)`).


## Emergency Patient Canadian Triage and Acuity Scale (CTAS) Level Breakdown by Zone for 2022 and 2023

In [23]:
# SQL query
query = """

SELECT
    Zone,
    CTAS,
    SUM(Actual) AS Actual,
    CONCAT(ROUND(SUM(Actual) * 100.0 / SUM(SUM(Actual)) OVER (PARTITION BY Zone), 2), '%') AS [Actual (Percent of Total)]
FROM dbo.Hospital_Service_Volumes
WHERE 
    CTAS IS NOT NULL AND 
    CTAS <> '0' AND 
    CTAS <> 'Blank' AND
    YEAR(Date) IN (2022, 2023) -- Filter for the years 2022 and 2023
GROUP BY Zone, CTAS
ORDER BY Zone, CTAS;
"""


try:
    df = pd.read_sql(query, engine)
    print(df)  
except Exception as e:
    print(f"Error executing SQL query: {str(e)}")

        Zone CTAS  Actual Actual (Percent of Total)
0    Central    1    4369           1.140000000000%
1    Central    2  103455          27.050000000000%
2    Central    3  173253          45.300000000000%
3    Central    4   94448          24.700000000000%
4    Central    5    6901           1.800000000000%
5    Eastern    1    1151           0.590000000000%
6    Eastern    2   34688          17.770000000000%
7    Eastern    3   76072          38.970000000000%
8    Eastern    4   72717          37.250000000000%
9    Eastern    5   10569           5.410000000000%
10       IWK    1     539           0.670000000000%
11       IWK    2   20054          24.960000000000%
12       IWK    3   28154          35.040000000000%
13       IWK    4   30610          38.090000000000%
14       IWK    5     996           1.240000000000%
15  Northern    1     911           0.450000000000%
16  Northern    2   32150          15.920000000000%
17  Northern    3   77868          38.560000000000%
18  Northern

### Explanation

- **SQL Query**: This SQL query retrieves data from the `Hospital_Service_Volumes` table and calculates the percentage of `Actual` values relative to the total `Actual` values for each `Zone` and `CTAS` category.

  - **Columns Selected**: 
    - `Zone`: Specifies the geographical zone.
    - `CTAS`: Represents the Canadian Triage and Acuity Scale category.
    - `SUM(Actual) AS Actual`: Calculates the total sum of `Actual` values for each `Zone` and `CTAS`.
    - `CONCAT(ROUND(SUM(Actual) * 100.0 / SUM(SUM(Actual)) OVER (PARTITION BY Zone), 2), '%') AS [Actual (Percent of Total)]`: Computes the percentage of `Actual` values relative to the total `Actual` values within each `Zone`.

  - **Filters**: 
    - `CTAS IS NOT NULL`: Ensures that the `CTAS` column is not null.
    - `CTAS <> '0'`: Excludes rows where `CTAS` is '0'.
    - `CTAS <> 'Blank'`: Excludes rows where `CTAS` is 'Blank'.
    - `YEAR(Date) IN (2022, 2023)`: Filters data for the years 2022 and 2023 based on the `Date` column.

  - **Grouping**: 
    - Groups the results by `Zone` and `CTAS` to calculate sums and percentages within each category.

  - **Ordering**: 
    - Orders the results first by `Zone` and then by `CTAS`.

- **Error Handling**: 
  - The `try` block attempts to execute the SQL query using Pandas `read_sql` function and displays the resulting DataFrame (`df`).
  - If an exception occurs during the execution (e.g., SQL query error), the `except` block catches the exception (`Exception as e`) and prints an error message indicating the issue (`Error executing SQL query: {str(e)}`).


## Number of Emergency Visits by Zone for 2022 and 2023

In [24]:
# SQL query
query = """

SELECT
    Zone,
    SUM(Actual) AS Number_of_Emergency_Visits
FROM dbo.Hospital_Service_Volumes
WHERE 
    Measure_Name = 'Emergency Visits' AND 
    YEAR(Date) IN (2022, 2023) -- Filter for the years 2022 and 2023
GROUP BY Zone;
"""


try:
    df = pd.read_sql(query, engine)
    print(df)  
except Exception as e:
    print(f"Error executing SQL query: {str(e)}")

       Zone  Number_of_Emergency_Visits
0  Northern                      202403
1   Eastern                      196456
2   Western                      282161
3       IWK                       81445
4   Central                      383267


### Explanation

- **SQL Query**: This SQL query retrieves data from the `Hospital_Service_Volumes` table to calculate the total number of emergency visits (`Actual`) grouped by `Zone` for the years 2022 and 2023.

  - **Columns Selected**: 
    - `Zone`: Represents the geographical zone.
    - `SUM(Actual) AS Number_of_Emergency_Visits`: Calculates the total sum of `Actual` values (number of emergency visits) for each `Zone`.

  - **Filters**: 
    - `Measure_Name = 'Emergency Visits'`: Filters rows where the `Measure_Name` column equals 'Emergency Visits'.
    - `YEAR(Date) IN (2022, 2023)`: Filters data for the years 2022 and 2023 based on the `Date` column.

  - **Grouping**: 
    - Groups the results by `Zone` to aggregate the total number of emergency visits within each zone.

## Trend of Emergency Visits by CTAS Level - Semi-Annual Data for 2022 and 2023

In [25]:
# SQL query
query = """

WITH SemiAnnualData AS (
    SELECT 
        Zone,
        CTAS,
        DATEPART(YEAR, Date) AS Year,
        CASE 
            WHEN DATEPART(MONTH, Date) BETWEEN 1 AND 6 THEN 'H1'
            WHEN DATEPART(MONTH, Date) BETWEEN 7 AND 12 THEN 'H2'
        END AS HalfYear,
        SUM(Actual) AS Total_Visits
    FROM 
        dbo.Hospital_Service_Volumes -- Replace with your actual table name
    WHERE 
        Measure_Name = 'Emergency Visits CTAS' 
        AND DATEPART(YEAR, Date) IN (2022, 2023)
        AND CTAS IS NOT NULL 
        AND CTAS <> 'Blank'
        AND CTAS <> 0
    GROUP BY 
        Zone,
        CTAS,
        DATEPART(YEAR, Date),
        CASE 
            WHEN DATEPART(MONTH, Date) BETWEEN 1 AND 6 THEN 'H1'
            WHEN DATEPART(MONTH, Date) BETWEEN 7 AND 12 THEN 'H2'
        END
)
SELECT 
    Zone,
    CTAS,
    Year,
    HalfYear,
    Total_Visits
FROM 
    SemiAnnualData
ORDER BY 
    Zone,
    CTAS,
    Year,
    HalfYear;
"""


try:
    df = pd.read_sql(query, engine)
    print(df)  
except Exception as e:
    print(f"Error executing SQL query: {str(e)}")

       Zone CTAS  Year HalfYear  Total_Visits
0   Central    1  2022       H1           944
1   Central    1  2022       H2          1108
2   Central    1  2023       H1          1081
3   Central    1  2023       H2          1236
4   Central    2  2022       H1         24720
..      ...  ...   ...      ...           ...
95  Western    4  2023       H2         26480
96  Western    5  2022       H1          4520
97  Western    5  2022       H2          3648
98  Western    5  2023       H1          3003
99  Western    5  2023       H2          2784

[100 rows x 5 columns]


### Explanation

- **SQL Query**: This SQL query retrieves data from the `Hospital_Service_Volumes` table to calculate the total number of emergency visits (`Actual`) grouped by `Zone`, `CTAS`, year, and half-year (H1 for January to June, H2 for July to December) for the years 2022 and 2023.

  - **Common Table Expression (CTE) - SemiAnnualData**:
    - Calculates the sum of `Actual` visits (`Total_Visits`) for each combination of `Zone`, `CTAS`, `Year`, and `HalfYear`.
    - Filters data where `Measure_Name` is 'Emergency Visits CTAS', `Date` year is either 2022 or 2023, and `CTAS` is not null, not 'Blank', and not equal to 0.
    - Uses `DATEPART` function to extract the year and determine the half-year based on the month of the `Date`.

  - **Main Query**:
    - Selects columns `Zone`, `CTAS`, `Year`, `HalfYear`, and `Total_Visits` from `SemiAnnualData`.
    - Orders the results by `Zone`, `CTAS`, `Year`, and `HalfYear` for clarity and organization.